# Install Rasterio

In [ ]:
! pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 45.4 MB/s eta 0:00:00


# Mount GDrive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import Library

In [ ]:
import geopandas as gpd
import rasterio
import numpy as np
import pandas as pd
from rasterio.mask import mask

# Load Data & Calculation

In [ ]:
import geopandas as gpd
import rasterio
import numpy as np
import pandas as pd
from rasterio.mask import mask

# Load shapefile desa
desa_gdf = gpd.read_file('/content/drive/MyDrive/Model Citra Malam Kemiskinan/SHP Desa/Lamsel/SHPdesaLamsel.shp')

# Load dataset DEM
with rasterio.open('/content/drive/MyDrive/Model Citra Malam Kemiskinan/Citra SRTM DEM/DEM SRTM 30M LAMPUNG.tif') as src:
    # Loop melalui setiap fitur desa dan hitung ketinggian rata-rata
    ketinggian_list = []
    for _, row in desa_gdf.iterrows():
        geom = row['geometry']
        if geom is not None and not geom.is_empty:
            # Masking DEM dengan geometri desa
            geom = [geom]
            try:
                out_image, out_transform = mask(src, geom, crop=True)

                # Menghilangkan nilai-nilai Nodata
                out_image = out_image[out_image != src.nodata]

                # Hitung ketinggian rata-rata jika data ada
                if len(out_image) > 0:
                    mean_elevation = out_image.mean()
                else:
                    mean_elevation = np.nan
            except Exception as e:
                print(f"Error processing geometry for row {row.name}: {e}")
                mean_elevation = np.nan
        else:
            print(f"Invalid or empty geometry for row {row.name}")
            mean_elevation = np.nan

        # Simpan ketinggian ke dalam list
        ketinggian_list.append(mean_elevation)

# Tambahkan kolom ketinggian ke GeoDataFrame
desa_gdf['ketinggian(m)'] = ketinggian_list

# Konversi ke Pandas DataFrame jika hanya ingin DataFrame
df = desa_gdf[['NAMOBJ', 'ketinggian(m)']].copy()

# Tampilkan DataFrame
df


Invalid or empty geometry for row 69


,NAMOBJ,ketinggian(m)
0,Babatan,170.383996
1,Babulang,303.360072
2,Agom,14.266709
3,Mekar Jaya,187.850877
4,Negeri Pandan,70.354973
...,...,...
251,Panca Tunggal,83.547060
252,Pancasila,69.514193
253,Purwodadi Dalam,77.044851
254,Purwodadi Simpang,60.553357


# Export to CSV

In [ ]:
df.to_csv('KetinggianDesaLamsel.csv')